In [1]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import numpy as np
from PIL import Image
from os import path

class ScenesDataset(Dataset):
    
    def __init__(self,base_path,txt_list,transform=None):
        self.base_path=base_path
        self.images = np.loadtxt(txt_list,dtype=str,delimiter=',')
        self.transform = transform
        
    def __getitem__(self, index):
        f,c = self.images[index]
        im = Image.open(path.join(self.base_path, f))
        if self.transform is not None:
            im = self.transform(im)
        
        label = init(c)
        
        return {'image' : im, 'label':label}
    
    def __len__(self):
        return len(self.images)

In [10]:
dataset = ScenesDataset('DITS-full\DITS-full\DITS-detection\class\image','DITS-full\DITS-full\DITS-detection\class\classes.txt',transform=transforms.ToTensor())
sample = dataset[0]
print(sample['image'].shape)
print(sample['label'])

NameError: name 'init' is not defined